In [1]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
from scipy.signal import stft, get_window
from scipy.stats import entropy
import os
import pickle

### FUNCTIONS

In [2]:
# Función para cargar el archivo .set y obtener los datos y la información
def cargar_eeg(filepath):
    raw = mne.io.read_raw_eeglab(filepath, preload=True)
    return raw

# Función para añadir canales vacíos al objeto Raw de MNE
def add_empty_channels(raw, target_n_channels=16):
    n_channels = len(raw.ch_names)
    if n_channels < target_n_channels:
        mean_channel = np.mean(raw.get_data(), axis=0)
        for i in range(n_channels + 1, target_n_channels + 1):
            ch_name = f'EEG {i}'
            info = mne.create_info(ch_names=[ch_name], sfreq=raw.info['sfreq'], ch_types='eeg')
            new_raw = mne.io.RawArray(mean_channel[np.newaxis, :], info)
            raw.add_channels([new_raw], force_update_info=True)

In [3]:
# Define frequency bands
bands = {
    'delta': (1, 4),
    'theta': (4, 8),
    'alpha': (8, 14),
    'beta': (14, 31),
    'gamma': (31, 50)
}

# Compute Differential Entropy from STFT
def compute_de_from_stft(segment, fs=125):
    window_size = int(fs * 1)  # window size of 1 second
    de_features = []
    for band, (low, high) in bands.items():
        # Compute the STFT with a Hanning window
        f, t, Zxx = scipy.signal.stft(segment, fs=fs, window='hann', nperseg=window_size, noverlap=0)
        
        # Select frequencies within the band
        band_indices = np.where((f >= low) & (f <= high))[0]
        if len(band_indices) == 0:
            continue
        
        band_power = np.abs(Zxx[band_indices, :])**2
        band_power_mean = np.mean(band_power)
        
        # Compute differential entropy
        de = 0.5 * np.log(2 * np.pi * np.e * band_power_mean)
        de_features.append(de)
    return np.array(de_features).flatten()

# FEATURE EXTRACTION

In [4]:
input_dir = '.../Preprocesados_SET'
output_dir = '.../Feature_Extraction'


# Listar todos los archivos .set en el directorio de entrada
for filename in os.listdir(input_dir):
    if filename.endswith('.set'):
        file_path = os.path.join(input_dir, filename)
        raw = cargar_eeg(file_path)
        add_empty_channels(raw)

        duration = 4
        epochs = mne.make_fixed_length_epochs(raw, duration=duration, preload=True, overlap=0)
        all_de_features = []

        for epoch_data in epochs.get_data():
            epoch_features = []
            for channel_data in epoch_data:
                de_features = compute_de_from_stft(channel_data, fs=raw.info['sfreq'])
                epoch_features.append(de_features)
            all_de_features.append(np.concatenate(epoch_features))

        all_de_features = np.array(all_de_features)

        # Guardar las características en un archivo
        output_file_path = os.path.join(output_dir, filename.replace('.set', '_features.pkl'))
        with open(output_file_path, 'wb') as f:
            pickle.dump(all_de_features, f)

        # Imprimir las dimensiones y un head de las primeras tres filas
        print(f"Processed {filename}: features saved to {output_file_path}")
        print("Shape of the features array:", all_de_features.shape)
        print(all_de_features[:1])

Not setting metadata
27 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 27 events and 500 original time points ...
0 bad epochs dropped


C:\Users\pablo\AppData\Local\Temp\ipykernel_9160\1625974722.py:3: RuntimeWarning: Limited 2 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(filepath, preload=True)
C:\Users\pablo\AppData\Local\Temp\ipykernel_9160\1625974722.py:3: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(filepath, preload=True)


Processed Emilio_1_seg1_preprocessed.set: features saved to C:/Users/pablo/OneDrive/Escritorio/UPV/TFG/Señales_Emilio/4.Feature_Extraction\Emilio_1_seg1_preprocessed_features.pkl
Shape of the features array: (27, 80)
[[-20.57563094 -21.07684031 -22.05475778 -22.48417143 -21.73399509
  -20.50649064 -21.09823571 -22.05255889 -22.51135752 -22.59962627
  -20.49846264 -21.11758551 -21.9869125  -22.41084209 -20.9605315
  -20.53779529 -21.05552835 -22.06519877 -22.51084092 -22.07759216
  -20.61272191 -21.80972295 -21.98516023 -21.99689895 -20.61170044
  -19.20177545 -21.00187035 -21.6346656  -21.92723325 -20.26196113
  -21.05118184 -22.59299912 -23.27703799 -23.67443178 -21.06780055
  -20.53238008 -20.95825779 -21.94319657 -22.33394432 -21.89382309
  -19.82089646 -20.82578306 -22.18324682 -22.76631557 -20.72494399
  -19.58735867 -20.22597408 -21.4302492  -21.8435158  -21.20217778
  -20.15272359 -20.79179634 -21.98996332 -22.46309515 -21.19046246
  -20.29541818 -20.82705895 -21.93837203 -22.37

C:\Users\pablo\AppData\Local\Temp\ipykernel_9160\1625974722.py:3: RuntimeWarning: Limited 2 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(filepath, preload=True)
C:\Users\pablo\AppData\Local\Temp\ipykernel_9160\1625974722.py:3: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(filepath, preload=True)


Processed Emilio_1_seg2_preprocessed.set: features saved to C:/Users/pablo/OneDrive/Escritorio/UPV/TFG/Señales_Emilio/4.Feature_Extraction\Emilio_1_seg2_preprocessed_features.pkl
Shape of the features array: (45, 80)
[[-16.38964947 -17.76938161 -18.29618443 -19.04010003 -18.23333042
  -15.8295611  -17.48941832 -17.95437051 -18.57076333 -16.9484494
  -15.20944659 -17.62967378 -18.23271516 -18.94153648 -19.00227938
  -15.84116912 -17.12396029 -17.80880029 -18.50459076 -18.48405796
  -16.13482388 -17.87394053 -18.62583215 -19.65997263 -18.84823737
  -15.77848352 -17.55102499 -18.09514157 -18.99385765 -18.63907318
  -15.50527681 -17.36942065 -17.99286352 -19.01311144 -18.07333028
  -16.10938852 -17.61229948 -18.3954563  -19.26002767 -18.09742813
  -15.98171833 -17.57348097 -18.16979884 -18.96612    -18.91034405
  -15.2641998  -17.12445474 -17.83096634 -18.60141566 -18.88876958
  -15.74000564 -17.70689549 -17.82346382 -18.97599145 -19.89095595
  -13.58111492 -15.20532227 -15.91229752 -16.75

...

Processed Emilio_5_seg3_preprocessed.set: features saved to C:/Users/pablo/OneDrive/Escritorio/UPV/TFG/Señales_Emilio/4.Feature_Extraction\Emilio_5_seg3_preprocessed_features.pkl
Shape of the features array: (45, 80)
[[-16.03091857 -17.61106908 -17.99035607 -18.36576658 -16.6810944
  -16.68768249 -18.14869408 -18.45390026 -18.81047096 -17.3810049
  -16.53310293 -17.85345112 -18.15357225 -18.41236173 -17.02547367
  -17.09942325 -18.600942   -19.20458006 -19.79359401 -18.01617965
  -16.62865666 -18.29012721 -18.79068772 -19.36458965 -17.36178669
  -16.28267857 -17.95127083 -18.34468903 -18.90799024 -17.11283008
  -17.36735859 -18.79874624 -19.32739897 -19.47709858 -17.74212233
  -17.19651254 -18.2796117  -18.68275724 -18.84758001 -18.56203273
  -17.03977859 -18.39766657 -18.73066822 -19.30908463 -17.38404098
  -16.68768248 -18.14869409 -18.45390024 -18.81047101 -17.38100509
  -15.82671476 -17.41338903 -17.81940456 -18.20437336 -16.46917447
  -17.52792853 -18.43489403 -18.79298967 -18.740

C:\Users\pablo\AppData\Local\Temp\ipykernel_9160\1625974722.py:3: RuntimeWarning: Limited 2 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(filepath, preload=True)
C:\Users\pablo\AppData\Local\Temp\ipykernel_9160\1625974722.py:3: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(filepath, preload=True)


Processed Emilio_5_seg4_preprocessed.set: features saved to C:/Users/pablo/OneDrive/Escritorio/UPV/TFG/Señales_Emilio/4.Feature_Extraction\Emilio_5_seg4_preprocessed_features.pkl
Shape of the features array: (37, 80)
[[-15.90237038 -16.96101928 -17.50143581 -17.7537416  -16.23405984
  -16.71253207 -17.84878776 -18.25835475 -18.49136633 -17.06077364
  -15.51940195 -17.31941719 -17.84880323 -18.31867657 -18.03339843
  -16.16402744 -17.34585386 -17.90798634 -18.20928386 -16.85094593
  -16.19433419 -17.20317027 -17.76644005 -18.0001766  -16.47244122
  -16.129398   -17.30898207 -17.80433547 -18.13007066 -16.61100998
  -16.55523971 -17.57275943 -18.08588305 -18.3450918  -16.73996887
  -16.43857617 -17.70112298 -18.28460419 -18.49531188 -17.06637583
  -16.0768071  -17.65663694 -18.36966377 -18.83785672 -18.23979981
  -16.66062497 -17.77979157 -18.2160315  -18.44778892 -17.01831375
  -16.60211008 -17.64033019 -18.0842673  -18.27271985 -16.89556053
  -16.17563392 -17.62077925 -18.36511665 -18.6

C:\Users\pablo\AppData\Local\Temp\ipykernel_9160\1625974722.py:3: RuntimeWarning: Limited 2 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(filepath, preload=True)
C:\Users\pablo\AppData\Local\Temp\ipykernel_9160\1625974722.py:3: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(filepath, preload=True)


Processed Emilio_5_seg5_preprocessed.set: features saved to C:/Users/pablo/OneDrive/Escritorio/UPV/TFG/Señales_Emilio/4.Feature_Extraction\Emilio_5_seg5_preprocessed_features.pkl
Shape of the features array: (37, 80)
[[-17.47183124 -18.35947441 -18.87002063 -19.4410758  -18.91786349
  -17.40028269 -18.58206098 -19.15413513 -19.80735325 -19.73349466
  -16.04996098 -17.28615816 -17.79709726 -18.44365169 -18.04518112
  -17.18820338 -18.2946508  -18.76049434 -18.99462428 -18.20807783
  -17.58245737 -18.82042571 -19.39373208 -19.11736252 -18.17464148
  -17.77465807 -18.81368805 -19.279488   -19.13672698 -18.19657427
  -17.4367298  -18.75762989 -19.36563928 -19.32226043 -18.30257966
  -17.40367514 -18.55741593 -19.14837761 -19.44985561 -18.97075796
  -16.28088491 -17.61481784 -18.59846174 -18.69006088 -17.83314255
  -17.40342116 -18.56151025 -19.15251176 -19.67262504 -19.17368913
  -18.65954288 -19.56509671 -19.94944087 -19.23627894 -18.30666029
  -17.4376383  -18.76981983 -19.4217982  -19.4

# AÑADIR ETIQUETAS. DATOS TEST

In [5]:
# Lista de emociones en el mismo orden siempre
emociones = ['Disgust', 'Fear', 'Sad', 'Neutral', 'Happy']

# Rutas de los archivos actualizadas
ruta_archivos = r'...\Feature_Extraction'
ruta_guardado = r'...\Labeled'

# Crear la ruta de guardado si no existe
os.makedirs(ruta_guardado, exist_ok=True)

# Función para procesar los archivos y agregar etiquetas
def agregar_etiqueta_a_archivos(ruta_lectura, ruta_escritura):
    for archivo in os.listdir(ruta_lectura):
        if archivo.endswith('.pkl'):
            # Extraer número de segmento del nombre del archivo
            partes = archivo.split('_')
            if len(partes) >= 3:
                segmento = partes[2]  # "seg1", "seg2", etc.
                numero_segmento = int(''.join(filter(str.isdigit, segmento)))  # Extraer el número
                
                # Cargar el archivo .pkl
                with open(os.path.join(ruta_lectura, archivo), 'rb') as f:
                    data = pickle.load(f)
                
                # Obtener la etiqueta correspondiente
                etiqueta = emociones[numero_segmento - 1]
                
                # Crear una columna de etiquetas
                etiqueta_columna = np.full((data.shape[0], 1), etiqueta, dtype=object)
                
                # Concatenar la columna de etiquetas al array de datos
                data_con_etiqueta = np.hstack((data, etiqueta_columna))
                
                # Guardar el nuevo archivo con la etiqueta añadida
                nuevo_nombre_archivo = os.path.join(ruta_escritura, archivo)
                with open(nuevo_nombre_archivo, 'wb') as f:
                    pickle.dump(data_con_etiqueta, f)
                print(f"Archivo procesado y guardado: {nuevo_nombre_archivo}")

# Ejecutar la función
agregar_etiqueta_a_archivos(ruta_archivos, ruta_guardado)

Archivo procesado y guardado: C:\Users\pablo\OneDrive\Escritorio\UPV\TFG\Señales_Emilio\5.Labeled\Emilio_1_seg1_preprocessed_features.pkl
Archivo procesado y guardado: C:\Users\pablo\OneDrive\Escritorio\UPV\TFG\Señales_Emilio\5.Labeled\Emilio_1_seg2_preprocessed_features.pkl
Archivo procesado y guardado: C:\Users\pablo\OneDrive\Escritorio\UPV\TFG\Señales_Emilio\5.Labeled\Emilio_1_seg3_preprocessed_features.pkl
Archivo procesado y guardado: C:\Users\pablo\OneDrive\Escritorio\UPV\TFG\Señales_Emilio\5.Labeled\Emilio_1_seg4_preprocessed_features.pkl
Archivo procesado y guardado: C:\Users\pablo\OneDrive\Escritorio\UPV\TFG\Señales_Emilio\5.Labeled\Emilio_1_seg5_preprocessed_features.pkl
Archivo procesado y guardado: C:\Users\pablo\OneDrive\Escritorio\UPV\TFG\Señales_Emilio\5.Labeled\Emilio_2_seg1_preprocessed_features.pkl
Archivo procesado y guardado: C:\Users\pablo\OneDrive\Escritorio\UPV\TFG\Señales_Emilio\5.Labeled\Emilio_2_seg2_preprocessed_features.pkl
Archivo procesado y guardado: C:\U

In [6]:
#COMPROBACION ETIQUETAS
import pandas as pd
import numpy as np
import pickle

def mostrar_valores_unicos_ultima_columna(ruta_archivo):
    # Abrir el archivo pickle y cargar el contenido
    with open(ruta_archivo, 'rb') as file:
        data = pickle.load(file)
    
    # Verificar si el contenido es un array de NumPy
    if isinstance(data, np.ndarray):
        # Asumir que la última columna contiene las etiquetas
        ultima_columna = data[:, -1]  # Extraer la última columna del array
        valores_unicos = np.unique(ultima_columna)  # Encontrar valores únicos
        
        # Mostrar los valores únicos
        print("Valores únicos en la última columna:")
        print(valores_unicos)
    else:
        print("El contenido no es un array de NumPy.")

# Uso del método
ruta_pickle = r'C:\Users\pablo\OneDrive\Escritorio\UPV\TFG\Señales_Emilio\5.Labeled\Emilio_1_seg5_preprocessed_features.pkl'
mostrar_valores_unicos_ultima_columna(ruta_pickle)

Valores únicos en la última columna:
['Happy']


# CONCATENAR Y EXPORTAR CSV

In [7]:
#ETIQUETAS A NUMERICO Y CONCATENAR

import tempfile

# Ruta de entrada de los archivos
ruta_entrada = r'...\Labeled'

# Usar una ruta temporal para la salida
ruta_salida = tempfile.gettempdir()
nombre_archivo = 'archivo_combinado.csv'
ruta_archivo_salida = os.path.join(ruta_salida, nombre_archivo)

# Diccionario de mapeo de etiquetas
label_mapping = {
    'Disgust': 0,
    'Fear': 1,
    'Sad': 2,
    'Neutral': 3,
    'Happy': 4
}

def procesar_archivos_y_combinar(ruta_entrada):
    dataframes = []
    
    for archivo in os.listdir(ruta_entrada):
        if archivo.endswith('.pkl'):
            ruta_completa = os.path.join(ruta_entrada, archivo)
            with open(ruta_completa, 'rb') as f:
                data = pickle.load(f)
            
            # Asumimos que las etiquetas están en la última columna
            etiquetas_numericas = np.vectorize(label_mapping.get)(data[:, -1].astype(str))
            
            # Reemplazamos la última columna con las nuevas etiquetas numéricas
            data[:, -1] = etiquetas_numericas
            
            # Convertimos el array a DataFrame
            df = pd.DataFrame(data)
            dataframes.append(df)
    
    # Combinar todos los DataFrames en uno solo
    df_combinado = pd.concat(dataframes, ignore_index=True)
    
    # Renombrar la última columna a 'Label'
    df_combinado.rename(columns={df_combinado.columns[-1]: 'Label'}, inplace=True)
    
    return df_combinado

# Llamar a la función y obtener el DataFrame combinado
df_final = procesar_archivos_y_combinar(ruta_entrada)

# Guardar el DataFrame final como CSV
df_final.to_csv(ruta_archivo_salida, index=False)

print("DataFrame combinado guardado en:", ruta_archivo_salida)

DataFrame combinado guardado en: C:\Users\pablo\AppData\Local\Temp\archivo_combinado.csv


In [11]:
#COMPROBACION CSV

# Ruta al archivo CSV generado
ruta_csv = r'...\Concatenacion\archivo_combinado.csv'

# Cargar el CSV en un DataFrame
df = pd.read_csv(ruta_csv)

# Obtener las dimensiones del DataFrame
dimensiones = df.shape
print("Dimensiones del DataFrame:", dimensiones)

# Obtener valores únicos de la última columna (asumimos que se llama 'Label')
valores_unicos = df['Label'].unique()
print("Valores únicos en la última columna 'Label':", valores_unicos)

Dimensiones del DataFrame: (731, 81)
Valores únicos en la última columna 'Label': [0 1 2 3 4]


In [12]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,79,Label
0,-11.079002,-11.534218,-12.353006,-12.588532,-12.044480,-10.442783,-10.749639,-11.040801,-11.113743,-11.025430,...,-11.301181,-11.485677,-11.533357,-11.532263,-11.081684,-11.539477,-12.361184,-12.597832,-12.061676,0
1,-10.274487,-10.715463,-11.777244,-12.146063,-11.751219,-9.379097,-9.938621,-10.722788,-11.024397,-11.049207,...,-10.552214,-11.334975,-11.381280,-11.349963,-10.273306,-10.719170,-11.803910,-12.155447,-11.764283,0
2,-8.025103,-8.557325,-9.686202,-10.774283,-10.566634,-8.367697,-9.291084,-9.827061,-10.740739,-10.727302,...,-8.474233,-9.593658,-10.689667,-10.416205,-8.028288,-8.557896,-9.685545,-10.774710,-10.571693,0
3,-6.885335,-7.739153,-8.545933,-9.226378,-9.588142,-8.459854,-9.221656,-10.051159,-10.627525,-10.643210,...,-7.784709,-8.587848,-9.275321,-9.582532,-6.885921,-7.739491,-8.546311,-9.226820,-9.589646,0
4,-5.805112,-7.322329,-8.117642,-9.012066,-9.529591,-7.834102,-8.840099,-9.586324,-10.306869,-10.478520,...,-7.356319,-8.156867,-9.046957,-9.535275,-5.804526,-7.322634,-8.117820,-9.012489,-9.530793,0
